In [ ]:
import ipywidgets as widgets
import traceback

from tiled.client import from_uri

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

TILED HELPER FUNCTIONS

In [ ]:
"""
    Variables to be declared at first
    tiled_base_uri - base uri
    additional_string_tiled_uri - the add on path to the base_uri, depends on file structure
    metadata_field - the field to extract
"""
tiled_base_uri = "http://127.0.0.1:8000"
additional_string_tiled_uri = "writable"
metadata_field = "job_name"

In [ ]:
# Gets the metadata entry in the dropdown for each scan with the tiled_uri for that scan
# In segmentation case, it gets job_name value and segmentation uri as (key,value) pairs
def get_tiled_uri_metadata_field_entries(base_uri, uri_additional, metadata_field):
    """
    base_uri - str, tiled base uri (ex: looks like http://127.0.0.1:8888)
    uri_additional - str, the add on path to the base_uri (ex: could be something like "node_name_1" or ""
                    based on your file structure)
    metadata_field - str, the field to extract
    """
    try:
        full_tiled_uri = base_uri + "/api/v1/metadata/" + uri_additional
        client = from_uri(full_tiled_uri)
        dropdown_list = []

        for experiment in client:
            if "seg_result" in client[experiment]:
                child_node_client = client[experiment]["seg_result"]
                scan_uri = child_node_client.uri
                metadata = child_node_client.metadata
                metadata_field_entry = metadata[metadata_field]
                dropdown_list.append((metadata_field_entry, scan_uri))

        return dropdown_list

    except Exception as e:
        error_message = (
            f"Error retrieving experiment names: {e} due to {traceback.format_exc()}"
        )
        print(error_message)
        return []

In [ ]:
def tiled_widget_refresh_callback(dropdown_widget):
    # should be the same string as the next cell, because the refresh fetches the same tiled uri
    new_list_experiments = get_tiled_uri_metadata_field_entries(
        tiled_base_uri, additional_string_tiled_uri, metadata_field
    )
    dropdown_widget.options = new_list_experiments
    dropdown_widget.value = None

In [ ]:
scan_uri_metadata_tuples = get_tiled_uri_metadata_field_entries(
    tiled_base_uri, additional_string_tiled_uri, metadata_field
)

tiled_file_widget = widgets.Dropdown(
    options=scan_uri_metadata_tuples,
    value=None,
    description="Experiment name:",
    style={"description_width": "initial"},
)

tiled_widget_refresh_button = widgets.Button(description="Refresh")

widget_box = widgets.HBox([tiled_file_widget, tiled_widget_refresh_button])

display(widget_box)


def tiled_refresh_button_click(button):
    tiled_widget_refresh_callback(tiled_file_widget)


tiled_widget_refresh_button.on_click(tiled_refresh_button_click)

In [ ]:
tiled_file_widget.value

Plot the segmentation result

In [ ]:
# With the selected job_name in the dropdown, you get the scan_uri
# Get the mask uri and plot the classes

In [ ]:
def plot_seg_mask(results_uri, slice_idx, axs):
    result_client = from_uri(results_uri)
    result_client_metadata = result_client.metadata
    mask_uri = result_client_metadata["mask_uri"]
    mask_uri = mask_uri[:-5]

    seg_result = result_client[slice_idx]

    # Extract class information
    mask_client = from_uri(mask_uri)
    mask_client_metadata = mask_client.metadata
    class_meta_data = mask_client_metadata["classes"]
    max_class_id = len(class_meta_data.keys()) - 1

    colors = [
        annotation_class["color"] for _, annotation_class in class_meta_data.items()
    ]
    labels = [
        annotation_class["label"] for _, annotation_class in class_meta_data.items()
    ]
    # Add color for unlabeled pixels
    colors = ["#D3D3D3"] + colors
    labels = ["Unlabeled"] + labels

    axs.imshow(
        seg_result,
        cmap=ListedColormap(colors),
        vmin=-1.5,
        vmax=max_class_id + 0.5,
    )
    axs.set_title(mask_client_metadata["project_name"] + ", slice: " + str(slice_idx))

    # create a patch for every color
    patches = [
        mpatches.Patch(color=colors[i], label=labels[i]) for i in range(len(labels))
    ]
    # Plot legend below the image
    axs.legend(
        handles=patches, loc="upper center", bbox_to_anchor=(0.5, -0.075), ncol=3
    )

In [ ]:
selected_scan_uri = tiled_file_widget.value

fig = plt.figure(layout="constrained")
plot_seg_mask(selected_scan_uri, 0, fig.gca())

# Alternative for multiple slices
# x_array = fig.subplots(2, 2, squeeze=False)
# plot_seg_mask(selected_scan_uri,0,ax_array[0, 0])
# plot_seg_mask(selected_scan_uri,1,ax_array[0, 1])
# plot_seg_mask(selected_scan_uri,2,ax_array[1, 0])
# plot_seg_mask(selected_scan_uri,3,ax_array[1, 1])